In [1]:
import Pkg;Pkg.activate(@__DIR__);Pkg.instantiate();

  Activating project at `~/Documents/eth_projects/freertos_sim/test/julia_cf2`


In [2]:
using GLMakie
using PyCall
GLMakie.activate!(inline=false)

In [11]:
points = Observable(Point2f[randn(2)])

fig, ax = scatter(points)
limits!(ax, -4, 4, -4, 4)

fps = 60
nframes = 1200

for i = 1:nframes
    new_point = Point2f(randn(2))
    points[] = push!(points[], new_point)
    sleep(1/fps) # refreshes the display!
end
display(fig)

GLMakie.Screen(...)

# CHannel demo

In [3]:
function producer(a)
    Channel() do ch
        put!(ch, "start")
        for n=1:4
            put!(ch, 2n)
        end
        put!(ch, "stop")
    end
end;

In [4]:
channel_countdown = producer(2)

Channel{Any}(0) (1 item available)

In [5]:
for i in channel_countdown
    println("countdown $i")
end

countdown start
countdown 2
countdown 4
countdown 6
countdown 8
countdown stop


In [6]:
function producer(c::Channel)

    put!(c, "start")
    for n=1:5
        put!(c, 2n)
    end
    put!(c, "stop")
end;

chnl = Channel(producer);

In [ ]:
# put!(chnl, "selva")

In [7]:
for n=1:5
    println(take!(chnl))
end

start
2
4
6
8


## new demo

In [6]:
# Given Channels c1 and c2,
c1 = Channel(32)
c2 = Channel(32)

# add data to c1
for i in 1:3
    put!(c1, i)
end

In [8]:
# and a function `foo` which reads items from c1, processes the item read
# and writes a result to c2,
function foo()
    for i in 1:3
        data = take!(c1)
        result = data*2               # process data
        put!(c2, result)    # write out result
    end
end

foo (generic function with 1 method)

## Condition demo

In [110]:
c = Condition()

@async try
    wait(c)
    println("got event")
    
catch
    println("failed")
end

Task (runnable) @0x000000010babb880

In [111]:

@async try
    notify(c)
    println("Notified")
    
catch
    println("failed")
end

Notified


Task (done) @0x000000010babbb20

got event


## Threads demo

In [37]:
Threads.nthreads()

4

In [38]:
Threads.threadid()

1

In [36]:
main_task = Threads.@spawn begin
    for i in 1:5
        println("selva")
    end
end


Task (runnable) @0x000000010b949cf0

selva
selva
selva
selva
selva


In [113]:

Threads.@spawn begin 
        Core.println("Thread-id $(Threads.threadid()), task 1")
        for i in 1:10
            Core.println("task 1: $(i)")
        end

end

Threads.@spawn begin 
        Core.println("Thread-id $(Threads.threadid()), task 2")
        for i in 1:10
            Core.println("task 2: $(i)")
        end

end


Thread-id 2, task 1
task 1: 1
task 1: 2
task 1: 3
task 1: 4
task 1: 5
task 1: 6
task 1: 7
task 1: 8
task 1: 9
task 1: 10
Thread-id 1, task 2
task 2: 1
task 2: 2
task 2: 3
task 2: 4
task 2: 5
task 2: 6
task 2: 7
task 2: 8
task 2: 9
task 2: 10


Task (done) @0x000000010badb1f0

# Threadpools

In [114]:
using ThreadPools

[ Info: Precompiling ThreadPools [b189fb0b-2eb5-4ed4-bc0c-d34c51242431]
